## SecSSE : Reproduction 

In [1]:
## SecSSE : Reproduction 

## I Data preparation

In [2]:
## I Data preparation

### Loading packages

In [3]:
### Loading packages

In [4]:
library("secsse")
library("stringr")
library("qpcR")
library("ape")
library("DDD")

Le chargement a nécessité le package : MASS

Le chargement a nécessité le package : minpack.lm

Le chargement a nécessité le package : rgl

This build of rgl does not include OpenGL functions.  Use
 rglwidget() to display results, e.g. via options(rgl.printRglwidget = TRUE).

Le chargement a nécessité le package : robustbase

Le chargement a nécessité le package : Matrix



### Loading data

In [5]:
### Loading data

In [6]:
df<-read.csv("Path/to/your/file.tsv", sep="\t") # omit sep ="\t" for .csv files
phy<-read.nexus("Path/to/your/tree.tree")

In [7]:
states<-df$Reproduction
names(states)=str_replace((rownames(df)), " ", "_")
states2<-states[!names(states) %in% setdiff(names(states), phy$tip.label)]

In [8]:
phy2.0<-drop.tip(phy, setdiff(phy$tip.label, (names(states))))

### Computing sampling fraction

In [9]:
### Computing sampling fraction

In [10]:
samp_frac<-c(
length(states2[states2=="O"])/length(df$Reproduction[df$Reproduction=="O"]),
length(states2[states2=="Oo"])/length(df$Reproduction[df$Reproduction=="Oo"]),
length(states2[states2=="PV"])/length(df$Reproduction[df$Reproduction=="PV"]),
length(states2[states2=="YV"])/length(df$Reproduction[df$Reproduction=="YV"]))

### Naming states according to Secsse

In [11]:
### Naming states according to Secsse

In [12]:
states3<-as.data.frame(cbind(names(states2), as.factor(states2)))
colnames(states3)<-c("species","states")
rownames(states3)<-NULL
traits <- sortingtraits(states3, phy2.0)

In [46]:
traits
states3 0.0358367783113309 0.0237273957947573 0.0423775017413011 0.040433164010602

[1] 4 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [38] 1 1 4 4 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 [75] 3 3 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 1 1 1 1 1 1 1 1 1 1 1 2
[112] 3 1 1 1 1 1 1 1 4 4 4 4 1 1 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
[149] 4 4 4 4 4 4 4 4 4 4 4 4 4 3 1 4 4 4 1 1 1 1 1 1 1 1 1 1 4 4 4 4 3 2 1 1 1
[186] 1 1 1 1 1 3 3 3 1 1 1 1 1 1 1 1 1 4 4 4 4 4 1 1 1 1 1 1 1 4 4 4 1 1 3 3 3
[223] 4 3 2 2 3 3 2 2 3 3 2 4 2 4 4 4 3 3 3 3 3 4 4 4 4 3 3 3 4 4 3 4 3 4 3 3 3
[260] 2 3 3 4 2 2 4 4 4 4 4 4 4 4 4 4 3 3 1 1 1 2 4 4 1 1 3 4 4 4 1 1 1 2 1 2 4
[297] 3 3 3 3 3 3 1 1 3 3 1 1 1 1 1 1 3 4 4 4 4 4 4 3 3 3 3 3 3 3 4 4 4 4 4 4 4
[334] 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4
[371] 3 4 4 4 4 4

species,states
<chr>,<chr>
Aculeola_nigra,4
Alopias_pelagicus,2
Alopias_superciliosus,2
Alopias_vulpinus,2
Apristurus_albisoma,1
Apristurus_ampliceps,1
Apristurus_aphyodes,1
Apristurus_australis,1
Apristurus_brunneus,1


## II Running the analysis

In [15]:
## II Running the analysis

### Initialisation

In [16]:
### Initialisation

In [17]:
startingpoint <- bd_ML(brts = ape::branching.times(phy2.0))
intGuessLambda <- startingpoint$lambda0
intGuessMu <- startingpoint$mu0
#COND=conditioning on the state of the root
cond<-"proper_cond"
#root_state_weight
root_state_weight<-"proper_weights"
#SAMPLING_FRACTION
#TOL
tol = c(1e-04, 1e-05, 1e-07)

You are optimizing lambda0 mu0 
You are fixing lambda1 mu1 
Optimizing the likelihood - this may take a while. 
The loglikelihood for the initial parameter values is -2042.37935685298.

Maximum likelihood parameter estimates: lambda0: 0.028186, mu0: 0.009724, lambda1: 0.000000, mu1: 0.000000: 
Maximum loglikelihood: -1780.702572


### Constant rate model

In [18]:
### Constant rate model

In [19]:
  idparslist <- id_paramPos(traits, num_concealed_states = 4)
  idparslist[[1]][] <- 1 #same speciation rate
  idparslist[[2]][] <- 2 #same extinction rate
  
  #same transition rate
  masterBlock<-matrix(99,ncol=4,nrow=4,byrow=T) 
  diag(masterBlock) <- NA
  masterBlock[1,2] <- 3
  masterBlock[1,3] <- 3
  masterBlock[1,4] <- 3
  masterBlock[2,1] <- 3
  masterBlock[2,3] <- 3
  masterBlock[2,4] <- 3
  masterBlock[3,1] <- 3
  masterBlock[3,2] <- 3
  masterBlock[3,4] <- 3
  masterBlock[4,1] <- 3
  masterBlock[4,2] <- 3
  masterBlock[4,3] <- 3
  diff.conceal <- TRUE
  myQ<-q_doubletrans(traits,masterBlock,diff.conceal)
  myQ<-replace(myQ, myQ == 4, 3)
  idparslist[[3]] <- myQ
  myQ

NA,3,3,3,3,0,0,0,3,0,0,0,3,0,0,0
3,NA,3,3,0,3,0,0,0,3,0,0,0,3,0,0
3,3,NA,3,0,0,3,0,0,0,3,0,0,0,3,0
3,3,3,NA,0,0,0,3,0,0,0,3,0,0,0,3
3,0,0,0,NA,3,3,3,3,0,0,0,3,0,0,0
0,3,0,0,3,NA,3,3,0,3,0,0,0,3,0,0
0,0,3,0,3,3,NA,3,0,0,3,0,0,0,3,0
0,0,0,3,3,3,3,NA,0,0,0,3,0,0,0,3
3,0,0,0,3,0,0,0,NA,3,3,3,3,0,0,0
0,3,0,0,0,3,0,0,3,NA,3,3,0,3,0,0
0,0,3,0,0,0,3,0,3,3,NA,3,0,0,3,0


In [20]:
#INITPARSOPT = initiale values of each parameters
initparsopt <- c(intGuessLambda, intGuessMu, 0.25) # speciation rate, extinction rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:3)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

CR_sp<-secsse_ml(phy2.0, traits, num_concealed_states=4, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2332.024 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0097240310612135 0.25 -2332.02363346129 initial 
2 0.0295954409554525 0.0097240310612135 0.25 -2330.20748314622 reflect 
3 0.0305371270580805 0.00923829728847183 0.23371335504886 -2329.34296379979 expand 
4 0.0305371270580805 0.00923829728847183 0.23371335504886 -2329.34296379979 reflect 
5 0.0318478857083907 0.00945412125781383 0.227493698235506 -2329.24524294836 reflect 
6 0.0318478857083907 0.00945412125781383 0.227493698235506 -2329.24524294836 contract inside 
7 0.030153269305312 0.010030103379914 0.215096245247148 -2328.79104846426 expand 
8 0.031707912610423 0.0097420301802166 0.192426539179104 -2327.66528037168 expand 
9 0.0326359426112582 0.0107510060156801 0.169382504288164 -2326.12763435597 expand 
10 0.0307990123109034 0.011617448

89 0.0347341156829601 0.0141425931218047 0.000383053151535377 -1882.79679456901 expand 
90 0.0347341156829601 0.0141425931218047 0.000383053151535377 -1882.79679456901 reflect 
91 0.0348736278281823 0.014191926355716 0.000383161570193256 -1882.79282721651 expand 
92 0.0348543105188432 0.0142254471989306 0.000383728419465559 -1882.78077593972 expand 
93 0.0348543105188432 0.0142254471989306 0.000383728419465559 -1882.78077593972 reflect 
94 0.0351725077696781 0.014361661325344 0.000380563177254687 -1882.7805403657 expand 
95 0.0351922512817005 0.0144327253825828 0.000377747962774785 -1882.76010953639 expand 
96 0.0353937916981531 0.0145609976554312 0.000382480555161796 -1882.75405748764 expand 
97 0.0353937916981531 0.0145609976554312 0.000382480555161796 -1882.75405748764 contract inside 
98 0.035294517669936 0.0146090210393763 0.000381098304107057 -1882.71575373825 expand 
99 0.035294517669936 0.0146090210393763 0.000381098304107057 -1882.71575373825 reflect 
100 0.0358376485399025 0.

### CTD2 model (Two conceal states)

In [21]:
### CTD2 model (Two conceal states)

In [22]:
  idparslist <- id_paramPos(traits, num_concealed_states = 2)
  idparslist[[1]][1:4] <- 1 # Speciation varies only between concealed states
  idparslist[[1]][5:8] <- 2 
  idparslist[[2]][] <- 3 # same extinction rate

  diag(idparslist[[3]]) <- NA # remove same state transition rate (diagonal)
  # same transition rates
  idparslist[[3]][1,c(2,3,4,5)] <- 4
  idparslist[[3]][1,c(6,7,8)] <- 0
  idparslist[[3]][2,c(1,3,4,6)] <- 4
  idparslist[[3]][2,c(5,7,8)] <- 0
  idparslist[[3]][3,c(1,2,4,7)] <- 4
  idparslist[[3]][3,c(5,6,8)] <- 0
  idparslist[[3]][4,c(1,2,3,8)] <- 4
  idparslist[[3]][4,c(5,6,7)] <- 0
  idparslist[[3]][5,c(1,6,7,8)] <- 4
  idparslist[[3]][5,c(2,3,4)] <- 0
  idparslist[[3]][6,c(2,5,7,8)] <- 4
  idparslist[[3]][6,c(1,3,4)] <- 0
  idparslist[[3]][7,c(3,5,6,8)] <- 4
  idparslist[[3]][7,c(1,2,4)] <- 0
  idparslist[[3]][8,c(4,5,6,7)] <- 4
  idparslist[[3]][8,c(1,2,3,8)] <- 0
  idparslist[[3]]

,1A,2A,3A,4A,1B,2B,3B,4B
1A,NA,4,4,4,4,0,0,0
2A,4,NA,4,4,0,4,0,0
3A,4,4,NA,4,0,0,4,0
4A,4,4,4,NA,0,0,0,4
1B,4,0,0,0,NA,4,4,4
2B,0,4,0,0,4,NA,4,4
3B,0,0,4,0,4,4,NA,4
4B,0,0,0,4,4,4,4,0


In [23]:
#INITPARSOPT = initiale values of each parameters
initparsopt <- c(rep(intGuessLambda,2), intGuessMu, 0.25)# speciation rate, extinction rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:4)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

CTD2_sp<-secsse_ml(phy2.0,traits, num_concealed_states=2, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2332.024 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0281861342432881 0.0097240310612135 0.25 -2332.02363346136 initial 
2 0.0295954409554525 0.0281861342432881 0.0097240310612135 0.25 -2330.95677236661 reflect 
3 0.0297718759860967 0.0297718759860967 0.00935968691417552 0.23020706455542 -2329.53154959388 expand 
4 0.0297718759860967 0.0297718759860967 0.00935968691417552 0.23020706455542 -2329.53154959388 reflect 
5 0.0304119601878212 0.0304119601878212 0.0101038360187992 0.21815166591286 -2328.74661339873 expand 
6 0.0304119601878212 0.0304119601878212 0.0101038360187992 0.21815166591286 -2328.74661339873 reflect 
7 0.0313791116049284 0.0317332662485427 0.00941659835393215 0.206735615875742 -2328.42653734326 reflect 
8 0.0323965765054638 0.0310680435695208 0.0102016820596631 0.173320748209796

74 0.0282309155416382 0.0461375815740351 0.0177496782848327 0.000448131372841313 -1878.10525170746 contract inside 
75 0.0282309155416382 0.0461375815740351 0.0177496782848327 0.000448131372841313 -1878.10525170746 contract inside 
76 0.0282309155416382 0.0461375815740351 0.0177496782848327 0.000448131372841313 -1878.10525170746 contract inside 
77 0.0282309155416382 0.0461375815740351 0.0177496782848327 0.000448131372841313 -1878.10525170746 contract outside 
78 0.0289301115107031 0.0456061329037137 0.0175271953692312 0.000444942316853163 -1878.08554081998 contract inside 
79 0.0289301115107031 0.0456061329037137 0.0175271953692312 0.000444942316853163 -1878.08554081998 contract inside 
80 0.0289301115107031 0.0456061329037137 0.0175271953692312 0.000444942316853163 -1878.08554081998 reflect 
81 0.0289301115107031 0.0456061329037137 0.0175271953692312 0.000444942316853163 -1878.08554081998 reflect 
82 0.0285378468984337 0.0459063825895849 0.0176529844719141 0.00045606498099699 -1878.0

148 0.0267726089162582 0.0430758637467873 0.0176901487515715 0.000455548806684755 -1877.62027809402 reflect 
149 0.0267675724587297 0.0430533842765356 0.0176879964284301 0.000455110314471548 -1877.62004365772 expand 
150 0.0267675724587297 0.0430533842765356 0.0176879964284301 0.000455110314471548 -1877.62004365772 reflect 
151 0.0267675724587297 0.0430533842765356 0.0176879964284301 0.000455110314471548 -1877.62004365772 contract outside 
152 0.0267675724587297 0.0430533842765356 0.0176879964284301 0.000455110314471548 -1877.62004365772 reflect 
153 0.0268195833957528 0.0430507393622655 0.0176755198078228 0.000455185843958836 -1877.61992703456 expand 
154 0.026826511683576 0.0431099321254828 0.0176811013452672 0.00045515652637578 -1877.61983239946 expand 
155 0.0268076233959757 0.0430217959237272 0.0176729983515896 0.000454554164415356 -1877.61945923296 expand 
156 0.0268141030715301 0.0430318531771053 0.0176727844514648 0.000453917415842155 -1877.61914542238 expand 
157 0.02681410307

224 0.0213006509319915 0.0402606447304473 0.0133477570540127 0.000523660450626096 -1877.03554459104 expand 
225 0.0214171581014212 0.0410764895937322 0.0137045343908551 0.00050820955361343 -1877.02551657079 reflect 
226 0.0200076877969733 0.0400354421347104 0.0121173166205486 0.000521645839212433 -1876.86704317904 expand 
227 0.0200076877969733 0.0400354421347104 0.0121173166205486 0.000521645839212433 -1876.86704317904 reflect 
228 0.0200076877969733 0.0400354421347104 0.0121173166205486 0.000521645839212433 -1876.86704317904 contract inside 
229 0.0208974079426281 0.0415968421613601 0.0126577922249545 0.000505653940341456 -1876.83685378627 expand 
230 0.0202753691093193 0.0403541837456297 0.0111634222078751 0.000534639233933257 -1876.58353117588 expand 
231 0.0202753691093193 0.0403541837456297 0.0111634222078751 0.000534639233933257 -1876.58353117588 reflect 
232 0.0202753691093193 0.0403541837456297 0.0111634222078751 0.000534639233933257 -1876.58353117588 reflect 
233 0.0193839750

297 0.0175409951416501 0.0385436905013731 0.00600897251556646 0.00049622875326238 -1876.05260549591 contract inside 
298 0.0175349648303945 0.0385433639001652 0.00600109378549933 0.000496138310656453 -1876.05260519935 contract inside 
299 0.0175349648303945 0.0385433639001652 0.00600109378549933 0.000496138310656453 -1876.05260519935 reflect 
300 0.017534821470628 0.0385394822363368 0.00600202336688464 0.000496094124002336 -1876.05260470817 contract inside 
301 0.017534821470628 0.0385394822363368 0.00600202336688464 0.000496094124002336 -1876.05260470817 contract inside 
302 0.017534821470628 0.0385394822363368 0.00600202336688464 0.000496094124002336 -1876.05260470817 contract inside 
303 0.0175388243751585 0.0385430026255712 0.00600566444726236 0.000496177751162203 -1876.05260468091 contract inside 
304 0.0175389453468598 0.0385416546991414 0.00600470170925766 0.000496135758461492 -1876.05260464109 contract outside 
305 0.0175370481998993 0.0385416029286902 0.00600500523372693 0.000

### CTD3 model (Three conceal states)

In [24]:
### CTD3 model (Three conceal states)

In [25]:
  idparslist <- id_paramPos(traits, num_concealed_states = 3)
  idparslist[[1]][1:4] <- 1 #Speciation varies only between concealed states
  idparslist[[1]][5:8] <- 2 
  idparslist[[1]][9:12] <- 3
  idparslist[[2]][] <- 4 #same extinction rate
  
  diag(idparslist[[3]]) <- NA#remove same state transition rate (diagonal)
  #all transition rates equal
  idparslist[[3]][1,c(2,3,4,5,9)] <- 5
  idparslist[[3]][1,c(6,7,8,10,11,12)] <- 0
  idparslist[[3]][2,c(1,3,4,6,10)] <- 5
  idparslist[[3]][2,c(5,7,8,9,11,12)] <- 0
  idparslist[[3]][3,c(1,2,4,7,11)] <- 5
  idparslist[[3]][3,c(5,6,8,9,10,12)] <- 0
  idparslist[[3]][4,c(1,2,3,8,12)] <- 5
  idparslist[[3]][4,c(5,6,7,9,10,11)] <- 0
  idparslist[[3]][5,c(1,6,7,8,9)] <- 5
  idparslist[[3]][5,c(2,3,4,10,11,12)] <- 0
  idparslist[[3]][6,c(2,5,7,8,10)] <- 5
  idparslist[[3]][6,c(1,3,4,9,11,12)] <- 0
  idparslist[[3]][7,c(3,5,6,8,11)] <- 5
  idparslist[[3]][7,c(1,2,4,9,10,12)] <- 0
  idparslist[[3]][8,c(4,5,6,7,12)] <- 5
  idparslist[[3]][8,c(1,2,3,9,10,11)] <- 0
  idparslist[[3]][9,c(1,5,10,11,12)] <- 5
  idparslist[[3]][9,c(2,3,4,6,7,8)] <- 0
  idparslist[[3]][10,c(2,6,9,11,12)] <- 5
  idparslist[[3]][10,c(1,3,4,5,7,8)] <- 0
  idparslist[[3]][11,c(3,7,9,10,12)] <- 5
  idparslist[[3]][11,c(1,2,4,5,6,8)] <- 0
  idparslist[[3]][12,c(4,8,9,10,11)] <- 5
  idparslist[[3]][12,c(1,2,3,5,6,7)] <- 0
  idparslist[[3]]

,1A,2A,3A,4A,1B,2B,3B,4B,1C,2C,3C,4C
1A,NA,5,5,5,5,0,0,0,5,0,0,0
2A,5,NA,5,5,0,5,0,0,0,5,0,0
3A,5,5,NA,5,0,0,5,0,0,0,5,0
4A,5,5,5,NA,0,0,0,5,0,0,0,5
1B,5,0,0,0,NA,5,5,5,5,0,0,0
2B,0,5,0,0,5,NA,5,5,0,5,0,0
3B,0,0,5,0,5,5,NA,5,0,0,5,0
4B,0,0,0,5,5,5,5,NA,0,0,0,5
1C,5,0,0,0,5,0,0,0,NA,5,5,5
2C,0,5,0,0,0,5,0,0,5,NA,5,5


In [26]:
#INITPARSOPT=initiale values of each parameters
initparsopt <- c(rep(intGuessLambda,3), intGuessMu, 0.25)#speciation rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:5)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

CTD3_sp<-secsse_ml(phy2.0,traits, num_concealed_states=3, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2332.024 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0097240310612135 0.25 -2332.02363346102 initial 
2 0.0290312549170369 0.0290312549170369 0.0290312549170369 0.00875303062128823 0.257470119521912 -2330.77186702124 expand 
3 0.0295389945138178 0.0295389945138178 0.0295389945138178 0.00914120660815997 0.230027279812938 -2329.68553036573 expand 
4 0.0295389945138178 0.0295389945138178 0.0295389945138178 0.00914120660815997 0.230027279812938 -2329.68553036573 reflect 
5 0.0295389945138178 0.0295389945138178 0.0295389945138178 0.00914120660815997 0.230027279812938 -2329.68553036573 reflect 
6 0.0295389945138178 0.0295389945138178 0.0295389945138178 0.00914120660815997 0.230027279812938 -2329.68553036573 reflect 
7 0.0295389945138178 0.0295389945138178 0.02953

63 0.0423325235117525 0.0348394667204913 0.0344714500120287 0.0117462217271677 0.000403215344828655 -1883.34525792407 contract inside 
64 0.0423325235117525 0.0348394667204913 0.0344714500120287 0.0117462217271677 0.000403215344828655 -1883.34525792407 contract outside 
65 0.0423784161630657 0.0349267537906708 0.0344405538050923 0.0117392422385046 0.000375769055228131 -1883.33494784917 contract inside 
66 0.0423784161630657 0.0349267537906708 0.0344405538050923 0.0117392422385046 0.000375769055228131 -1883.33494784917 reflect 
67 0.0423839335752757 0.034893183084136 0.0344181613839251 0.0117520659339119 0.000413368517879363 -1883.31470464318 contract inside 
68 0.0423839335752757 0.034893183084136 0.0344181613839251 0.0117520659339119 0.000413368517879363 -1883.31470464318 contract outside 
69 0.0423839335752757 0.034893183084136 0.0344181613839251 0.0117520659339119 0.000413368517879363 -1883.31470464318 contract inside 
70 0.0423557613574191 0.0349184000282643 0.0344217017376044 0.01

126 0.0504373561445714 0.0449131924960654 0.0255225014684929 0.0126381500362076 0.000449677843316289 -1874.56391600425 expand 
127 0.0504373561445714 0.0449131924960654 0.0255225014684929 0.0126381500362076 0.000449677843316289 -1874.56391600425 reflect 
128 0.0504318371742921 0.0449369627337227 0.0255175276599413 0.0126354931392143 0.000455249942748469 -1874.56333325525 expand 
129 0.0504514898563901 0.0449526987047929 0.0255059244399437 0.0126360219696345 0.000449497813028693 -1874.56314379066 reflect 
130 0.0505468612744082 0.0450694026806473 0.0254132012076403 0.0126430199757607 0.000451694327767695 -1874.5624551856 reflect 
131 0.0505468612744082 0.0450694026806473 0.0254132012076403 0.0126430199757607 0.000451694327767695 -1874.5624551856 reflect 
132 0.0503985390563341 0.0448664780456737 0.0255912201100052 0.0126242791429799 0.000450008450601484 -1874.56200036679 reflect 
133 0.0504687705728062 0.0450041232496191 0.0255061808816717 0.0126255046426022 0.000455500701820184 -1874.5

191 0.0772277608884297 0.0709790099474415 0.0271491330998017 0.00638125782958518 0.000331383988789284 -1872.44820136834 reflect 
192 0.0771607738886208 0.0715392330062943 0.0257548197078749 0.00676486847187014 0.00037605568371188 -1872.28970240191 expand 
193 0.0771607738886208 0.0715392330062943 0.0257548197078749 0.00676486847187014 0.00037605568371188 -1872.28970240191 reflect 
194 0.0771607738886208 0.0715392330062943 0.0257548197078749 0.00676486847187014 0.00037605568371188 -1872.28970240191 reflect 
195 0.082365307789978 0.0765009487225677 0.0263405472224341 0.00551511203662426 0.000347401431990633 -1872.25150058167 contract outside 
196 0.0799752660461128 0.0740708727277796 0.0265496372906536 0.00594958608164188 0.000340912846260465 -1872.23330539514 contract inside 
197 0.0799752660461128 0.0740708727277796 0.0265496372906536 0.00594958608164188 0.000340912846260465 -1872.23330539514 reflect 
198 0.0786807880886225 0.0726866999990524 0.026626045345669 0.00621035764473957 0.000

253 0.0800382287024114 0.0742665749897111 0.0260720158400196 0.00607730986374583 0.00036076364435606 -1872.13079129713 reflect 
254 0.0800382287024114 0.0742665749897111 0.0260720158400196 0.00607730986374583 0.00036076364435606 -1872.13079129713 reflect 
255 0.0800095167132283 0.0742376713256064 0.0260747075590167 0.00608242009440846 0.000361066466991209 -1872.1307870078 reflect 
256 0.0799911300366632 0.0742193183053439 0.0260758674652612 0.00608587170461184 0.000360941256961334 -1872.13078560248 reflect 
257 0.0800078406187178 0.0742367361196966 0.0260722816672631 0.00608348733597103 0.000360808811890467 -1872.13078433552 reflect 
258 0.0800078406187178 0.0742367361196966 0.0260722816672631 0.00608348733597103 0.000360808811890467 -1872.13078433552 contract inside 
259 0.0800037983369381 0.074235603453107 0.0260706725326329 0.00608437904478707 0.000360913167828231 -1872.13077364578 expand 
260 0.0800037983369381 0.074235603453107 0.0260706725326329 0.00608437904478707 0.000360913167

317 0.0773327324669231 0.0755448866657126 0.0259508589486105 0.00599175781743012 0.000367720492873999 -1872.11941430494 reflect 
318 0.0752073874656222 0.0760229581162836 0.0259638810392754 0.00597941380523261 0.000370480366131388 -1872.11816465007 expand 
319 0.0753109254479881 0.0760359652476485 0.0259578820569265 0.00597562769608608 0.0003683708806416 -1872.11570254145 expand 
320 0.0753109254479881 0.0760359652476485 0.0259578820569265 0.00597562769608608 0.0003683708806416 -1872.11570254145 reflect 
321 0.0753109254479881 0.0760359652476485 0.0259578820569265 0.00597562769608608 0.0003683708806416 -1872.11570254145 reflect 
322 0.0730388219651942 0.0774195361566046 0.0258619273959078 0.00584934889477995 0.000373419640523108 -1872.10749444499 expand 
323 0.0730388219651942 0.0774195361566046 0.0258619273959078 0.00584934889477995 0.000373419640523108 -1872.10749444499 reflect 
324 0.0730388219651942 0.0774195361566046 0.0258619273959078 0.00584934889477995 0.000373419640523108 -187

381 0.0435001077416445 0.0785323714491874 0.0253427775328813 0.00691911163105049 0.000404139873930167 -1869.98720985548 contract inside 
382 0.0433101335133221 0.0818013709203121 0.0252718058687283 0.00638630824019263 0.000407369507053535 -1869.98638941126 contract inside 
383 0.0419283586875379 0.0788951263760328 0.0252986983936681 0.00693093723972076 0.000406311193232269 -1869.93075152448 expand 
384 0.0418695985834012 0.0804217631078324 0.0251864369606883 0.00670783893005591 0.00041305341079482 -1869.87133883126 expand 
385 0.0418695985834012 0.0804217631078324 0.0251864369606883 0.00670783893005591 0.00041305341079482 -1869.87133883126 reflect 
386 0.0418695985834012 0.0804217631078324 0.0251864369606883 0.00670783893005591 0.00041305341079482 -1869.87133883126 reflect 
387 0.0418695985834012 0.0804217631078324 0.0251864369606883 0.00670783893005591 0.00041305341079482 -1869.87133883126 reflect 
388 0.0410223797355489 0.0798867506119065 0.0251373360924746 0.00684927527780935 0.0004

444 0.0414034672858796 0.081172821984212 0.0241195687958681 0.00696311758943878 0.000477401097340385 -1869.58097650412 reflect 
445 0.0415729001388394 0.0824978418157181 0.0240707186244587 0.00674545975089679 0.000477902457074963 -1869.57440922547 expand 
446 0.0415729001388394 0.0824978418157181 0.0240707186244587 0.00674545975089679 0.000477902457074963 -1869.57440922547 reflect 
447 0.0413862609582288 0.0816416069293792 0.024157428401639 0.00687004905142841 0.000472069861044914 -1869.56353494716 expand 
448 0.0413862609582288 0.0816416069293792 0.024157428401639 0.00687004905142841 0.000472069861044914 -1869.56353494716 reflect 
449 0.0417913112451205 0.0815204176035423 0.0239990378900379 0.0069302541079692 0.000478827667684049 -1869.55613506355 expand 
450 0.0414415465083195 0.083121085097048 0.023958968844186 0.00668208355688518 0.000478486428898527 -1869.54481296543 expand 
451 0.0414415465083195 0.083121085097048 0.023958968844186 0.00668208355688518 0.000478486428898527 -1869.5

507 0.0399467959826884 0.0862538458118848 0.0223356602969379 0.00676789930152283 0.000439620778507199 -1869.16531931368 contract inside 
508 0.0399467959826884 0.0862538458118848 0.0223356602969379 0.00676789930152283 0.000439620778507199 -1869.16531931368 reflect 
509 0.0399467959826884 0.0862538458118848 0.0223356602969379 0.00676789930152283 0.000439620778507199 -1869.16531931368 contract inside 
510 0.0399467959826884 0.0862538458118848 0.0223356602969379 0.00676789930152283 0.000439620778507199 -1869.16531931368 contract inside 
511 0.0399467959826884 0.0862538458118848 0.0223356602969379 0.00676789930152283 0.000439620778507199 -1869.16531931368 contract inside 
512 0.0399542807973368 0.0862459663364086 0.0223485046251834 0.00676450371602646 0.000439270142134224 -1869.16530199912 reflect 
513 0.0399542807973368 0.0862459663364086 0.0223485046251834 0.00676450371602646 0.000439270142134224 -1869.16530199912 contract inside 
514 0.0399542807973368 0.0862459663364086 0.0223485046251

569 0.0399581226681198 0.0862503582217707 0.0223428060288582 0.00676560283942452 0.000438959074195499 -1869.16528960189 contract inside 
570 0.0399578184833834 0.0862505224276902 0.022342763733967 0.00676560100704253 0.000438961173389777 -1869.16528960166 contract inside 
571 0.0399578955133471 0.0862507775741681 0.0223427798564308 0.0067655489433278 0.000438959970949053 -1869.16528960157 contract inside 
572 0.0399578955133471 0.0862507775741681 0.0223427798564308 0.0067655489433278 0.000438959970949053 -1869.16528960157 contract outside 
573 0.0399578955133471 0.0862507775741681 0.0223427798564308 0.0067655489433278 0.000438959970949053 -1869.16528960157 reflect 
574 0.0399578955133471 0.0862507775741681 0.0223427798564308 0.0067655489433278 0.000438959970949053 -1869.16528960157 reflect 
575 0.0399578955133471 0.0862507775741681 0.0223427798564308 0.0067655489433278 0.000438959970949053 -1869.16528960157 reflect 
576 0.0399579513742693 0.0862504857267339 0.0223428027518471 0.0067655

### CTD4 model (Four conceal states)

In [27]:
### CTD4 model (Four conceal states)

In [28]:
  idparslist <- id_paramPos(traits, num_concealed_states = 4)
  idparslist[[1]][1:4] <- 1 #Speciation varies only between concealed states
  idparslist[[1]][5:8] <- 2 
  idparslist[[1]][9:12] <- 3  
  idparslist[[1]][13:16] <- 4 
  idparslist[[2]][] <- 5 #same extinction rate
  
  masterBlock<-matrix(99,ncol=4,nrow=4,byrow=T) 
  diag(masterBlock) <- NA
  masterBlock[1,2] <- 6
  masterBlock[1,3] <- 6
  masterBlock[1,4] <- 6
  masterBlock[2,1] <- 6
  masterBlock[2,3] <- 6
  masterBlock[2,4] <- 6
  masterBlock[3,1] <- 6
  masterBlock[3,2] <- 6
  masterBlock[3,4] <- 6
  masterBlock[4,1] <- 6
  masterBlock[4,2] <- 6
  masterBlock[4,3] <- 6
  diff.conceal <- TRUE
  myQ<-q_doubletrans(traits,masterBlock,diff.conceal)
  myQ<-replace(myQ, myQ == 7, 6)
  idparslist[[3]] <- myQ
  myQ

NA,6,6,6,6,0,0,0,6,0,0,0,6,0,0,0
6,NA,6,6,0,6,0,0,0,6,0,0,0,6,0,0
6,6,NA,6,0,0,6,0,0,0,6,0,0,0,6,0
6,6,6,NA,0,0,0,6,0,0,0,6,0,0,0,6
6,0,0,0,NA,6,6,6,6,0,0,0,6,0,0,0
0,6,0,0,6,NA,6,6,0,6,0,0,0,6,0,0
0,0,6,0,6,6,NA,6,0,0,6,0,0,0,6,0
0,0,0,6,6,6,6,NA,0,0,0,6,0,0,0,6
6,0,0,0,6,0,0,0,NA,6,6,6,6,0,0,0
0,6,0,0,0,6,0,0,6,NA,6,6,0,6,0,0
0,0,6,0,0,0,6,0,6,6,NA,6,0,0,6,0


In [29]:
#INITPARSOPT=initiale values of each parameters
initparsopt <- c(rep(intGuessLambda,4), intGuessMu, 0.25)#speciation rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:6)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

CTD4_sp<-secsse_ml(phy2.0,traits, num_concealed_states=4, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2332.024 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0097240310612135 0.25 -2332.02363346129 initial 
2 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.00875303062128823 0.256218905472637 -2330.8739695158 expand 
3 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.00923829728847183 0.228710462287105 -2329.8916955468 expand 
4 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.00923829728847183 0.228710462287105 -2329.8916955468 reflect 
5 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.00923829728847183 0.228710462287105 -2329.8916955468 reflect 
6 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.

56 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.0338566182995387 0.0108082598412025 0.000380243770558153 -1882.79116967508 contract inside 
57 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.0338566182995387 0.0108082598412025 0.000380243770558153 -1882.79116967508 contract inside 
58 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.0338566182995387 0.0108082598412025 0.000380243770558153 -1882.79116967508 contract inside 
59 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.0338566182995387 0.0108082598412025 0.000380243770558153 -1882.79116967508 contract inside 
60 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.0338566182995387 0.0108082598412025 0.000380243770558153 -1882.79116967508 contract inside 
61 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.0338566182995387 0.0108082598412025 0.000380243770558153 -1882.79116967508 contract inside 
62 0.0394957653031858 0.0403545013302134 0.0330115536034002 0.03385661829953

111 0.0472496560773956 0.0310062589111433 0.030451917728859 0.0279949299958433 0.0171751093689267 0.000452960444774385 -1877.48690004488 contract inside 
112 0.0472037897134596 0.0310341715485314 0.0304641381111052 0.028001935003615 0.0171707011944538 0.00047232078267434 -1877.48083781982 contract inside 
113 0.0472037897134596 0.0310341715485314 0.0304641381111052 0.028001935003615 0.0171707011944538 0.00047232078267434 -1877.48083781982 contract outside 
114 0.0476836189633641 0.0304729977192541 0.0303113271645953 0.0276477244509468 0.0175525421690984 0.000470027630527861 -1877.47992496848 contract inside 
115 0.0476836189633641 0.0304729977192541 0.0303113271645953 0.0276477244509468 0.0175525421690984 0.000470027630527861 -1877.47992496848 reflect 
116 0.0475170397744551 0.0306728853978806 0.0303616837966477 0.0277794652362944 0.0174176473275967 0.000460543765147552 -1877.47796457139 contract inside 
117 0.0474122274992126 0.0308150477941497 0.0303941389195722 0.0278812489389882 0.

167 0.0451655163219691 0.031463059902773 0.0315193838530509 0.0267141308016186 0.0177198673125874 0.000467459871970761 -1877.32277674731 reflect 
168 0.0451655163219691 0.031463059902773 0.0315193838530509 0.0267141308016186 0.0177198673125874 0.000467459871970761 -1877.32277674731 reflect 
169 0.0451655163219691 0.031463059902773 0.0315193838530509 0.0267141308016186 0.0177198673125874 0.000467459871970761 -1877.32277674731 contract outside 
170 0.0451655163219691 0.031463059902773 0.0315193838530509 0.0267141308016186 0.0177198673125874 0.000467459871970761 -1877.32277674731 contract outside 
171 0.0451655163219691 0.031463059902773 0.0315193838530509 0.0267141308016186 0.0177198673125874 0.000467459871970761 -1877.32277674731 contract outside 
172 0.0451655163219691 0.031463059902773 0.0315193838530509 0.0267141308016186 0.0177198673125874 0.000467459871970761 -1877.32277674731 reflect 
173 0.0451655163219691 0.031463059902773 0.0315193838530509 0.0267141308016186 0.0177198673125874

223 0.0442583785170396 0.0321030734677973 0.0319119167281588 0.0267604405651447 0.017459592671317 0.000452443044807806 -1877.30410175062 reflect 
224 0.0442583785170396 0.0321030734677973 0.0319119167281588 0.0267604405651447 0.017459592671317 0.000452443044807806 -1877.30410175062 reflect 
225 0.0442583785170396 0.0321030734677973 0.0319119167281588 0.0267604405651447 0.017459592671317 0.000452443044807806 -1877.30410175062 reflect 
226 0.0442583785170396 0.0321030734677973 0.0319119167281588 0.0267604405651447 0.017459592671317 0.000452443044807806 -1877.30410175062 reflect 
227 0.0441758221047487 0.0322120737435129 0.0319380562940214 0.0268355425990343 0.0173791231830978 0.000451994840651284 -1877.30378924428 expand 
228 0.0442711055153029 0.0320900687729619 0.0319121793929344 0.0267484654785979 0.0174630934525806 0.000451329689957374 -1877.30329055302 expand 
229 0.0440632711208747 0.0322292621123082 0.0320060249471695 0.0267455634235425 0.0174095364365426 0.000450902649659909 -187

...

498 0.0665126742506676 0.0237829536739637 0.0387352560303467 0.0150103489946771 6.64188511357528e-09 0.000478957836187745 -1866.70920073513 reflect 
499 0.0665081774267799 0.023785522078233 0.0387363915725118 0.0150109118565418 4.69810984593316e-09 0.000479092650902314 -1866.70919943135 reflect 
500 0.0665081774267799 0.023785522078233 0.0387363915725118 0.0150109118565418 4.69810984593316e-09 0.000479092650902314 -1866.70919943135 reflect 
501 0.0665164944842068 0.0237791463661652 0.0387365808765817 0.0150053759657366 7.96989896081532e-09 0.000479272443664073 -1866.70919751044 expand 
502 0.0665237678471956 0.0237762549438002 0.0387329505024413 0.0150079630389797 1.6290668115207e-09 0.000478998385663961 -1866.70919644629 reflect 
503 0.0665237678471956 0.0237762549438002 0.0387329505024413 0.0150079630389797 1.6290668115207e-09 0.000478998385663961 -1866.70919644629 contract inside 
504 0.0665237678471956 0.0237762549438002 0.0387329505024413 0.0150079630389797 1.6290668115207e-09 0.0

552 0.0665242519851123 0.0237763505598062 0.0387322763728727 0.0150089497891141 2.3089385079841e-10 0.000479200161034593 -1866.70919294639 reflect 
553 0.0665242519851123 0.0237763505598062 0.0387322763728727 0.0150089497891141 2.3089385079841e-10 0.000479200161034593 -1866.70919294639 contract inside 
554 0.0665242519851123 0.0237763505598062 0.0387322763728727 0.0150089497891141 2.3089385079841e-10 0.000479200161034593 -1866.70919294639 contract inside 
555 0.0665242519851123 0.0237763505598062 0.0387322763728727 0.0150089497891141 2.3089385079841e-10 0.000479200161034593 -1866.70919294639 contract inside 
556 0.0665242519851123 0.0237763505598062 0.0387322763728727 0.0150089497891141 2.3089385079841e-10 0.000479200161034593 -1866.70919294639 contract inside 
557 0.066524002138929 0.0237764517613092 0.0387323993154915 0.0150088653086872 4.35599108716889e-11 0.00047920453362598 -1866.70919288957 reflect 
558 0.066524002138929 0.0237764517613092 0.0387323993154915 0.0150088653086872 4.

607 0.0665279620340942 0.0237749296133878 0.0387303286606079 0.0150104337566709 9.10825240056849e-13 0.000479222832496474 -1866.7091927041 reflect 
608 0.0665279620340942 0.0237749296133878 0.0387303286606079 0.0150104337566709 9.10825240056849e-13 0.000479222832496474 -1866.7091927041 reflect 
609 0.0665279620340942 0.0237749296133878 0.0387303286606079 0.0150104337566709 9.10825240056849e-13 0.000479222832496474 -1866.7091927041 reflect 
610 0.0665279620340942 0.0237749296133878 0.0387303286606079 0.0150104337566709 9.10825240056849e-13 0.000479222832496474 -1866.7091927041 reflect 
611 0.0665279620340942 0.0237749296133878 0.0387303286606079 0.0150104337566709 9.10825240056849e-13 0.000479222832496474 -1866.7091927041 reflect 
612 0.0665279620340942 0.0237749296133878 0.0387303286606079 0.0150104337566709 9.10825240056849e-13 0.000479222832496474 -1866.7091927041 contract inside 
613 0.0665276005840563 0.023775112961239 0.0387304543100989 0.0150104139517108 2.87748705237754e-12 0.00

661 0.066527689496159 0.0237750634730045 0.0387304298073343 0.0150104065350859 4.31758983679216e-14 0.000479217994669025 -1866.70919270206 contract inside 
662 0.066527689496159 0.0237750634730045 0.0387304298073343 0.0150104065350859 4.31758983679216e-14 0.000479217994669025 -1866.70919270206 contract inside 
663 0.066527689496159 0.0237750634730045 0.0387304298073343 0.0150104065350859 4.31758983679216e-14 0.000479217994669025 -1866.70919270206 contract inside 
664 0.066527689496159 0.0237750634730045 0.0387304298073343 0.0150104065350859 4.31758983679216e-14 0.000479217994669025 -1866.70919270206 reflect 
665 0.066527689496159 0.0237750634730045 0.0387304298073343 0.0150104065350859 4.31758983679216e-14 0.000479217994669025 -1866.70919270206 contract inside 
666 0.066527689496159 0.0237750634730045 0.0387304298073343 0.0150104065350859 4.31758983679216e-14 0.000479217994669025 -1866.70919270206 contract inside 
667 0.066527689496159 0.0237750634730045 0.0387304298073343 0.0150104065

716 0.0665276801525056 0.0237750659877683 0.0387304362371693 0.0150103998376064 2.33798795604048e-15 0.000479217946184981 -1866.70919270204 reflect 
717 0.0665276801525056 0.0237750659877683 0.0387304362371693 0.0150103998376064 2.33798795604048e-15 0.000479217946184981 -1866.70919270204 shrink 
718 0.0665276801525056 0.0237750659877683 0.0387304362371693 0.0150103998376064 2.33798795604048e-15 0.000479217946184981 -1866.70919270204 reflect 
719 0.0665276801525056 0.0237750659877683 0.0387304362371693 0.0150103998376064 2.33798795604048e-15 0.000479217946184981 -1866.70919270204 reflect 
720 0.0665276801525056 0.0237750659877683 0.0387304362371693 0.0150103998376064 2.33798795604048e-15 0.000479217946184981 -1866.70919270204 contract inside 
721 0.0665276801525056 0.0237750659877683 0.0387304362371693 0.0150103998376064 2.33798795604048e-15 0.000479217946184981 -1866.70919270204 contract inside 
722 0.0665276801525056 0.0237750659877683 0.0387304362371693 0.0150103998376064 2.337987956

### ETD model (Focal trait)

In [30]:
### ETD model (Focal trait)

In [31]:
 idparslist <- id_paramPos(traits, num_concealed_states = 4)
  idparslist[[1]][c(1,5,9,13)] <- 1 #Speciation varies only between the 4 states
  idparslist[[1]][c(2,6,10,14)] <- 2 
  idparslist[[1]][c(3,7,11,15)] <- 3
  idparslist[[1]][c(4,8,12,16)] <- 4
  idparslist[[2]][] <- 5 #same extinction rate
  
  #all transition rates equal
  masterBlock<-matrix(99,ncol=4,nrow=4,byrow=T) 
  diag(masterBlock) <- NA
  masterBlock[1,2] <- 6
  masterBlock[1,3] <- 6
  masterBlock[1,4] <- 6
  masterBlock[2,1] <- 6
  masterBlock[2,3] <- 6
  masterBlock[2,4] <- 6
  masterBlock[3,1] <- 6
  masterBlock[3,2] <- 6
  masterBlock[3,4] <- 6
  masterBlock[4,1] <- 6
  masterBlock[4,2] <- 6
  masterBlock[4,3] <- 6
  diff.conceal <- TRUE
  myQ<-q_doubletrans(traits,masterBlock,diff.conceal)
  myQ<-replace(myQ, myQ == 7, 6)
  idparslist[[3]] <- myQ
  myQ

NA,6,6,6,6,0,0,0,6,0,0,0,6,0,0,0
6,NA,6,6,0,6,0,0,0,6,0,0,0,6,0,0
6,6,NA,6,0,0,6,0,0,0,6,0,0,0,6,0
6,6,6,NA,0,0,0,6,0,0,0,6,0,0,0,6
6,0,0,0,NA,6,6,6,6,0,0,0,6,0,0,0
0,6,0,0,6,NA,6,6,0,6,0,0,0,6,0,0
0,0,6,0,6,6,NA,6,0,0,6,0,0,0,6,0
0,0,0,6,6,6,6,NA,0,0,0,6,0,0,0,6
6,0,0,0,6,0,0,0,NA,6,6,6,6,0,0,0
0,6,0,0,0,6,0,0,6,NA,6,6,0,6,0,0
0,0,6,0,0,0,6,0,6,6,NA,6,0,0,6,0


In [32]:
#INITPARSOPT=initiale values of each parameters
initparsopt <- c(rep(intGuessLambda,4), intGuessMu, 0.25)#speciation rate, transition rate
#IDPARSOPT = parameters we want to optimise
idparsopt <- c(1:6)
#IDPARSFIX = parameters we want to fix
idparsfix <- 0
#PARSFIX = values of the fix parameters
parsfix<- 0

ETD_sp<-secsse_ml(phy2.0,traits, num_concealed_states=4, idparslist,#here the matrix
          idparsopt, initparsopt, idparsfix, parsfix,
          cond=cond,root_state_weight = root_state_weight, tol = tol,
          sampling_fraction=samp_frac, 
          maxiter = 1000 * round((1.25)^length(idparsopt)), 
          use_fortran=TRUE,methode="ode45", optimmethod = "simplex", 
          num_cycles = 1, run_parallel=FALSE)

You set some transitions as impossible to happen 
Calculating the likelihood for the initial parameters. 
The loglikelihood for the initial parameter values is -2332.024 
Optimizing the likelihood - this may take a while. 
 
1 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0281861342432881 0.0097240310612135 0.25 -2332.02363346129 initial 
2 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.0288903050055341 0.00875303062128823 0.256218905472637 -2330.8739695158 expand 
3 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.00923829728847183 0.228710462287105 -2329.8916955468 expand 
4 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.00923829728847183 0.228710462287105 -2329.8916955468 reflect 
5 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.00923829728847183 0.228710462287105 -2329.8916955468 reflect 
6 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.0292427522080496 0.

55 0.0289032899506411 0.0379176910282866 0.033651337540226 0.0484858447663455 0.00960692828165143 0.000234154330201023 -1902.44795758603 contract inside 
56 0.0289032899506411 0.0379176910282866 0.033651337540226 0.0484858447663455 0.00960692828165143 0.000234154330201023 -1902.44795758603 reflect 
57 0.0289032899506411 0.0379176910282866 0.033651337540226 0.0484858447663455 0.00960692828165143 0.000234154330201023 -1902.44795758603 contract inside 
58 0.0289032899506411 0.0379176910282866 0.033651337540226 0.0484858447663455 0.00960692828165143 0.000234154330201023 -1902.44795758603 contract inside 
59 0.0289032899506411 0.0379176910282866 0.033651337540226 0.0484858447663455 0.00960692828165143 0.000234154330201023 -1902.44795758603 contract inside 
60 0.0289032899506411 0.0379176910282866 0.033651337540226 0.0484858447663455 0.00960692828165143 0.000234154330201023 -1902.44795758603 contract inside 
61 0.0289032899506411 0.0379176910282866 0.033651337540226 0.0484858447663455 0.0096

110 0.0289023835647466 0.0379058472759522 0.0336445095988529 0.0484620124347852 0.00961498568142241 0.000415459331687861 -1900.09597935098 expand 
111 0.0289023835647466 0.0379058472759522 0.0336445095988529 0.0484620124347852 0.00961498568142241 0.000415459331687861 -1900.09597935098 reflect 
112 0.0289023835647466 0.0379058472759522 0.0336445095988529 0.0484620124347852 0.00961498568142241 0.000415459331687861 -1900.09597935098 reflect 
113 0.0289023835647466 0.0379058472759522 0.0336445095988529 0.0484620124347852 0.00961498568142241 0.000415459331687861 -1900.09597935098 reflect 
114 0.0291075032569872 0.0379082889115844 0.0334884656427935 0.0484464268979867 0.00963181936908545 0.000416545343200852 -1900.04741510524 expand 
115 0.0291075032569872 0.0379082889115844 0.0334884656427935 0.0484464268979867 0.00963181936908545 0.000416545343200852 -1900.04741510524 reflect 
116 0.0291075032569872 0.0379082889115844 0.0334884656427935 0.0484464268979867 0.00963181936908545 0.000416545343

166 0.0295703482467754 0.0342114598559198 0.0353078117529382 0.0497642231854195 0.00962869334303539 0.000418453436938475 -1899.1368893866 expand 
167 0.0295703482467754 0.0342114598559198 0.0353078117529382 0.0497642231854195 0.00962869334303539 0.000418453436938475 -1899.1368893866 reflect 
168 0.0297320253158166 0.0328753515947798 0.0359762013220518 0.0502396488478642 0.00962755106377817 0.000438300360404192 -1899.01510583993 expand 
169 0.030667180185411 0.0325585334164456 0.0354602059081127 0.0502977389663597 0.00969927238449212 0.0003877603990545 -1898.87015396893 expand 
170 0.030667180185411 0.0325585334164456 0.0354602059081127 0.0502977389663597 0.00969927238449212 0.0003877603990545 -1898.87015396893 reflect 
171 0.030667180185411 0.0325585334164456 0.0354602059081127 0.0502977389663597 0.00969927238449212 0.0003877603990545 -1898.87015396893 reflect 
172 0.030667180185411 0.0325585334164456 0.0354602059081127 0.0502977389663597 0.00969927238449212 0.0003877603990545 -1898.87

221 0.0312393847850498 0.0313360700761064 0.0357582187816517 0.0506948522918782 0.00973845116222938 0.000411082879415163 -1898.76445627619 expand 
222 0.0312393847850498 0.0313360700761064 0.0357582187816517 0.0506948522918782 0.00973845116222938 0.000411082879415163 -1898.76445627619 reflect 
223 0.0312393847850498 0.0313360700761064 0.0357582187816517 0.0506948522918782 0.00973845116222938 0.000411082879415163 -1898.76445627619 reflect 
224 0.0311974808137957 0.0315826359665196 0.0356440483913398 0.0506007450509597 0.00974084585201895 0.000424351626408316 -1898.75300934961 expand 
225 0.0310980294353577 0.0308813466646906 0.0361368647242299 0.0508643273601247 0.00972772107776504 0.000417037551771971 -1898.74617328053 expand 
226 0.0310980294353577 0.0308813466646906 0.0361368647242299 0.0508643273601247 0.00972772107776504 0.000417037551771971 -1898.74617328053 reflect 
227 0.0315370861769496 0.030906319413753 0.0357959359181477 0.0508181126322692 0.00976708704356535 0.00041082094264

...

442 0.0358467185146335 0.023757224669629 0.0424667931509272 0.0403898999803803 0.0183826874238687 0.000394130568182156 -1875.92005716273 contract inside 
443 0.0358467185146335 0.023757224669629 0.0424667931509272 0.0403898999803803 0.0183826874238687 0.000394130568182156 -1875.92005716273 reflect 
444 0.0358467185146335 0.023757224669629 0.0424667931509272 0.0403898999803803 0.0183826874238687 0.000394130568182156 -1875.92005716273 contract outside 
445 0.0358467185146335 0.023757224669629 0.0424667931509272 0.0403898999803803 0.0183826874238687 0.000394130568182156 -1875.92005716273 reflect 
446 0.0358466516181245 0.0237570736357908 0.0424669333161738 0.040389965162021 0.0183826771697351 0.000394129916560297 -1875.92005716085 expand 
447 0.0358466516181245 0.0237570736357908 0.0424669333161738 0.040389965162021 0.0183826771697351 0.000394129916560297 -1875.92005716085 reflect 
448 0.0358466516181245 0.0237570736357908 0.0424669333161738 0.040389965162021 0.0183826771697351 0.00039412

499 0.0358480202367187 0.0237542924814842 0.0424680786431641 0.0403894527250029 0.0183838073708021 0.000394118336247866 -1875.92005573554 reflect 
500 0.0358494176024118 0.0237526986185038 0.0424682393636625 0.0403891823312355 0.0183844662474632 0.000394090581700666 -1875.92005555139 expand 
501 0.0358494176024118 0.0237526986185038 0.0424682393636625 0.0403891823312355 0.0183844662474632 0.000394090581700666 -1875.92005555139 reflect 
502 0.0358493079632272 0.0237516248676031 0.0424690509508577 0.0403892509202193 0.0183847026733099 0.000394135159484651 -1875.9200550459 expand 
503 0.0358493079632272 0.0237516248676031 0.0424690509508577 0.0403892509202193 0.0183847026733099 0.000394135159484651 -1875.9200550459 reflect 
504 0.0358493079632272 0.0237516248676031 0.0424690509508577 0.0403892509202193 0.0183847026733099 0.000394135159484651 -1875.9200550459 reflect 
505 0.0358493079632272 0.0237516248676031 0.0424690509508577 0.0403892509202193 0.0183847026733099 0.000394135159484651 -18

556 0.0358552679251278 0.0237587489505173 0.042456252523035 0.0403805683617375 0.0183926383129828 0.00039377799232791 -1875.92001260622 reflect 
557 0.0358557040303678 0.0237498096964723 0.0424606711079291 0.0403821779390908 0.0183944591641502 0.000393801117132295 -1875.91999727523 expand 
558 0.0358557040303678 0.0237498096964723 0.0424606711079291 0.0403821779390908 0.0183944591641502 0.000393801117132295 -1875.91999727523 reflect 
559 0.0358557040303678 0.0237498096964723 0.0424606711079291 0.0403821779390908 0.0183944591641502 0.000393801117132295 -1875.91999727523 reflect 
560 0.0358557040303678 0.0237498096964723 0.0424606711079291 0.0403821779390908 0.0183944591641502 0.000393801117132295 -1875.91999727523 reflect 
561 0.0358546006684497 0.0237640349549462 0.0424486109216705 0.0403807559683527 0.0183934552698419 0.000393698139602772 -1875.91997794778 expand 
562 0.0358546006684497 0.0237640349549462 0.0424486109216705 0.0403807559683527 0.0183934552698419 0.000393698139602772 -1

612 0.0358280863946177 0.0237356181300853 0.0423857740215628 0.0404325500985925 0.0184099373080189 0.000393931818297272 -1875.91945692345 expand 
613 0.0358280863946177 0.0237356181300853 0.0423857740215628 0.0404325500985925 0.0184099373080189 0.000393931818297272 -1875.91945692345 reflect 
614 0.0358280863946177 0.0237356181300853 0.0423857740215628 0.0404325500985925 0.0184099373080189 0.000393931818297272 -1875.91945692345 contract inside 
615 0.0358280863946177 0.0237356181300853 0.0423857740215628 0.0404325500985925 0.0184099373080189 0.000393931818297272 -1875.91945692345 reflect 
616 0.0358253510993201 0.0237406833225331 0.042379567583085 0.0404298334839092 0.0184135279501697 0.000394200950803098 -1875.91945511968 reflect 
617 0.0358307176885597 0.023743343455484 0.0423759087564108 0.040429681797349 0.0184118292942898 0.000393976407949728 -1875.91945444533 reflect 
618 0.0358307176885597 0.023743343455484 0.0423759087564108 0.040429681797349 0.0184118292942898 0.000393976407949

667 0.0358364163454751 0.0237274601733649 0.0423778716480878 0.040433178203206 0.0184157339351459 0.000394252852631742 -1875.91943890898 contract inside 
668 0.0358366221851606 0.0237274592385897 0.0423771429665548 0.0404332662102641 0.0184159733819627 0.000394249548825963 -1875.91943890811 contract inside 
669 0.0358366221851606 0.0237274592385897 0.0423771429665548 0.0404332662102641 0.0184159733819627 0.000394249548825963 -1875.91943890811 reflect 
670 0.0358366221851606 0.0237274592385897 0.0423771429665548 0.0404332662102641 0.0184159733819627 0.000394249548825963 -1875.91943890811 contract inside 
671 0.0358370264165577 0.0237271605504388 0.0423774262724704 0.0404332719341521 0.0184158484795756 0.000394255578069656 -1875.91943890214 contract inside 
672 0.0358370264165577 0.0237271605504388 0.0423774262724704 0.0404332719341521 0.0184158484795756 0.000394255578069656 -1875.91943890214 contract inside 
673 0.0358370264165577 0.0237271605504388 0.0423774262724704 0.0404332719341521

### Saving Data

In [33]:
### Saving Data

In [34]:
save(CR_sp, CTD2_sp, CTD3_sp, CTD4_sp, ETD_sp, file = "reprod_data.rds")

### Comparing models

In [35]:
### Comparing models

In [36]:
# Aicc function
aicc<-function(ll,k){round((2*(-round(ll,4))+2*k+(2*k*(k+1))/(Ntip(phy2.0)-k-1)),3)}

In [37]:
aicc_vec<-c(aicc(CR_sp$ML, 3), aicc(CTD2_sp$ML, 4), aicc(CTD3_sp$ML, 5), aicc(CTD4_sp$ML, 6), aicc(ETD_sp$ML, 6))

In [38]:
reprod_model_comp<-as.data.frame(cbind(c(CR_sp$ML, CTD2_sp$ML, CTD3_sp$ML, CTD4_sp$ML, ETD_sp$ML), c(3, 4, 5, 6, 6), aicc_vec, round(akaike.weights(aicc_vec)$deltaAIC, 3), round(akaike.weights(aicc_vec)$weights, 3)))

In [39]:
colnames(reprod_model_comp)<-c("log likelihood", "number of parameters", "AICc", "Delta AIC", "AIC weights")
rownames(reprod_model_comp)<-c("Constant rate model", "Concealt trait 2", "Conceal trait 3", "Conceal trait 4", "Focal trait")

### Saving Data

In [40]:
### Saving Data

In [43]:
write.table(reprod_model_comp, "Path/to/your/exit.file", sep ="\t")